In [15]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import matplotlib.pyplot as plt



Matplotlib is building the font cache; this may take a moment.


## Zeitserie erstellen

In [2]:
def generate_time_eries(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # Welle 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # Welle 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # Rauschen
    return series[..., np.newaxis].astype(np.float32)

In [4]:
n_steps = 50
series = generate_time_eries(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1541 - val_loss: 0.0570
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.0423 - val_loss: 0.0264
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - loss: 0.0211 - val_loss: 0.0148
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 0.0133 - val_loss: 0.0101
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0093 - val_loss: 0.0081
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 0.0077 - val_loss: 0.0070
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - loss: 0.0068 - val_loss: 0.0061
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.0060 - val_loss: 0.0054
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 12/20
219/219 ━━━━━━━━━━━━

### Das einfachste RNN das man bauen könnte

In [11]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


/Users/riccardo/Desktop/Github/Bachelor/Bachelor/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2590 - val_loss: 0.1882
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1740 - val_loss: 0.1522
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1483 - val_loss: 0.1414
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1418 - val_loss: 0.1439
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1403 - val_loss: 0.1420
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1397 - val_loss: 0.1467
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1380 - val_loss: 0.1406
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1424 - val_loss: 0.1408
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1413 - val_loss: 0.1407
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1402 - val_loss: 0.1415
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1371 - val_loss: 0.1405
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.

---------------------------------------------------------------------------

# Mehrere Zeitschritte vorhersagen

In [17]:
series = generate_time_eries(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

y_pred = X[:, n_steps:]




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [1]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [5]:
import pandas as pd

In [14]:
df = pd.read_csv("data/raw/^GDAXI.csv")
df = df.dropna()
df.columns



Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [18]:
X = df.drop(columns=['Open'])

# Create the target variable 'y' as the 'Open' column
y = df['Open']
X
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


,Date,High,Low,Close,Adj Close,Volume
0,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,0.0
3,1988-01-04,956.489990,956.489990,956.489990,956.489990,0.0
4,1988-01-05,996.099976,996.099976,996.099976,996.099976,0.0
5,1988-01-06,1006.010010,1006.010010,1006.010010,1006.010010,0.0
6,1988-01-07,1014.469971,1014.469971,1014.469971,1014.469971,0.0
...,...,...,...,...,...,...
9376,2024-06-03,18697.089844,18503.789063,18608.160156,18608.160156,54181600.0
9377,2024-06-04,18608.300781,18365.529297,18405.640625,18405.640625,81414800.0
9378,2024-06-05,18624.359375,18406.060547,18575.939453,18575.939453,63882100.0
9379,2024-06-06,18784.650391,18576.580078,18652.669922,18652.669922,60878800.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Dense, Embedding

from sklearn.preprocessing import MinMaxScaler

# Beispiel-Daten
data = np.sin(np.linspace(0, 100, 1000))  # Sinuswelle als Beispiel-Zeitreihe

# Daten skalieren
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.reshape(-1, 1))

# Sequenzen erstellen
sequence_length = 50
X, y = [], []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i + sequence_length])
    y.append(scaled_data[i + sequence_length])
X, y = np.array(X), np.array(y)

# Modell erstellen und trainieren
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10


/Users/riccardo/Desktop/Github/Bachelor/Bachelor/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.2135
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0122
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 8.4820e-04
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.3031e-04
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.6555e-05
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.0599e-05
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.4161e-05
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.3389e-05
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.3729e-05
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.9445e-05
